Converting features into ones we can actually use, and moving the target value to the last column.

In [28]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('data_with_target.csv')


In [ ]:
df.head()

In [ ]:
from datetime import datetime
epoch = pd.to_datetime(0, unit='s').to_julian_date()
epoch

In [27]:
 #convert the 4 columns in time_cols into datetime objects
# then calculate the timedelta 
def make_time(row):
    # pad leading 0s to front of time so 438 = 0438, etc
    disc_time = str(row['DISCOVERY_TIME']).zfill(4)
    disc_time = datetime.strptime(disc_time, '%H%M.%S').time()
    cont_time = str(row['CONT_TIME']).zfill(4)
    cont_time = datetime.strptime(cont_time, '%H%M.%S').time()
    # dates are formatted as Julian, convert to datetime
    disc_date = pd.to_datetime(row['DISCOVERY_DATE'] - epoch, unit='D')
    cont_date = pd.to_datetime(row['CONT_DATE'] - epoch, unit='D')
    # combine the time and the date
    disc = datetime.combine(disc_date, disc_time)
    cont = datetime.combine(cont_date, cont_time)
    delta = cont-disc 
    return delta

df['TIME_TO_CONT'] = df.apply(lambda row: make_time(row), axis=1)

ValueError: ("time data '1300' does not match format '%H%M.%S'", 'occurred at index 0')

In [ ]:
df.head()

In [ ]:
df_features = df.pop('TIME_TO_CONT')